In [1]:
#! /usr/bin/env python
"""
Compute elevation statistics for the debris-covered areas in each latitude and longitude
"""

import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
from scipy import ndimage
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib

import debrisglobal.globaldebris_input as debris_prms

In [2]:
# Debris cover extent shapefile with statistics
dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
dc_shp = dc_shp.sort_values(by=['RGIId'])

print('All DC glaciers:', dc_shp.shape[0], 'All DC Area (km2):', dc_shp.DC_Area_v2.sum() / 1e6)

# Subset by percent debris-covered or debris-covered area
dc_shp_subset = dc_shp[((dc_shp['DC_Area__1'] > debris_prms.dc_percarea_threshold) | 
                        (dc_shp['DC_Area_v2'] / 1e6 > debris_prms.dc_area_threshold))
                        & (dc_shp['Area'] > debris_prms.min_glac_area)].copy()
dc_shp_subset.reset_index(inplace=True, drop=True)

rgino_str_list_subset = [x.split('-')[1] for x in dc_shp_subset.RGIId.values]

print('Subset DC glaciers:', dc_shp_subset.shape[0], 'Subset DC Area (km2):', dc_shp_subset.DC_Area_v2.sum() / 1e6)

dc_shp_subset

All DC glaciers: 3313 All DC Area (km2): 448.801022
Subset DC glaciers: 219 Subset DC Area (km2): 154.25134


,RGIId,GLIMSId,BgnDate,EndDate,CenLon,CenLat,O1Region,O2Region,Area,Zmin,...,Name,DC_Area,DC_BgnDate,DC_EndDate,DC_CTSmean,DC_Area_%,area_singl,DC_Area_v2,DC_Area__1,geometry
0,RGI60-02.00280,G237427E49886N,20049999,20069999,-122.573034,49.882732,2,2,5.938,1671,...,None,468900,2013,2017,35.192442,7.90,21617,304435,5.127,"MULTIPOLYGON (((-122.60226 49.89003, -122.5997..."
1,RGI60-02.00737,G243472E50214N,20049999,20069999,-116.530729,50.214035,2,3,5.536,2237,...,None,450900,2013,2017,38.927991,8.14,288223,288223,5.206,"POLYGON ((-116.53549 50.23111, -116.53339 50.2..."
2,RGI60-02.00914,G236143E50316N,20049999,20069999,-123.854172,50.314732,2,2,4.372,1056,...,None,873900,2013,2017,16.054972,19.99,28820,784568,17.945,"MULTIPOLYGON (((-123.86658 50.31353, -123.8657..."
3,RGI60-02.01104,G237395E50420N,20049999,20069999,-122.607486,50.418684,2,2,3.016,1850,...,Place Glacier,269100,2013,2017,35.039186,8.92,169332,169332,5.614,"POLYGON ((-122.60957 50.43131, -122.60957 50.4..."
4,RGI60-02.01152,G236110E50341N,20049999,20069999,-123.869526,50.360763,2,2,12.021,1280,...,None,1129500,2013,2017,20.501797,9.40,517865,680881,5.664,"MULTIPOLYGON (((-123.87865 50.35121, -123.8778..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,RGI60-02.18765,G238984E48311N,19589999,-9999999,-121.017198,48.311120,2,4,4.845,1523,...,Chickamin Glacier WA,554400,2013,2017,59.437023,11.44,103528,485905,10.029,"MULTIPOLYGON (((-121.00731 48.31688, -121.0069..."
215,RGI60-02.18770,G238996E48072N,19849999,-9999999,-121.003616,48.073581,2,4,2.125,1845,...,Butterfly Glacier WA,265500,2013,2017,59.737753,12.49,32408,114329,5.380,"MULTIPOLYGON (((-121.00635 48.07096, -121.0059..."
216,RGI60-02.18778,G238943E48357N,19589999,-9999999,-121.057928,48.354310,2,4,2.924,1613,...,South Cascade Glacier WA,879300,2013,2017,56.691099,30.07,805510,805510,27.548,"POLYGON ((-121.06724 48.35866, -121.06683 48.3..."
217,RGI60-02.18805,G250362E43173N,19669999,-9999999,-109.639270,43.172392,2,5,3.061,3328,...,Dinwoody Glacier WY,766800,2013,2017,23.246983,25.05,695596,695596,22.724,"POLYGON ((-109.64516 43.17897, -109.64515 43.1..."


In [3]:
# Select glaciers using RGI and find unique latlons
#  (Scherler DC shapefiles do not have same CenLat and CenLon for some reason)
main_glac_rgi_subset = debris_prms.selectglaciersrgitable(rgino_str_list_subset)
main_glac_rgi_subset['CenLon_360'] = main_glac_rgi_subset['CenLon']
main_glac_rgi_subset.loc[main_glac_rgi_subset['CenLon_360'] < 0, 'CenLon_360'] = (
    360 + main_glac_rgi_subset.loc[main_glac_rgi_subset['CenLon_360'] < 0, 'CenLon_360'])

# Load met data and find nearest latlon indices
ds = xr.open_dataset(debris_prms.metdata_fp + '../' + debris_prms.metdata_elev_fn)
#  argmin() finds the minimum distance between the glacier lat/lon and the GCM pixel
lat_nearidx = (np.abs(main_glac_rgi_subset['CenLat'].values[:,np.newaxis] - 
                      ds['latitude'][:].values).argmin(axis=1))
lon_nearidx = (np.abs(main_glac_rgi_subset['CenLon_360'].values[:,np.newaxis] - 
                      ds['longitude'][:].values).argmin(axis=1))
latlon_nearidx = list(zip(lat_nearidx, lon_nearidx))
latlon_nearidx_unique = sorted(list(set(latlon_nearidx)))
main_glac_rgi_subset['latlon_nearidx'] = latlon_nearidx
latlon_unique_dict = dict(zip(latlon_nearidx_unique,np.arange(0,len(latlon_nearidx_unique))))
latlon_unique_dict_reversed = dict(zip(np.arange(0,len(latlon_nearidx_unique)),latlon_nearidx_unique))
main_glac_rgi_subset['latlon_unique_no'] = main_glac_rgi_subset['latlon_nearidx'].map(latlon_unique_dict)

print('unique lat/lons:', len(np.unique(main_glac_rgi_subset['latlon_unique_no'])), '\n\n')

# Delete me
latlon_nearidx_unique_v1 = latlon_nearidx_unique.copy()

lat_list = np.array([ds.latitude[x[0]].values for x in latlon_nearidx_unique])
lon_list = np.array([ds.longitude[x[1]].values for x in latlon_nearidx_unique])
latlon_list = list(tuple(zip(list(lat_list), list(lon_list))))

# Pickle unique lat/lons that will be used for melt model
with open(debris_prms.latlon_unique_fp + debris_prms.latlon_unique_dict[debris_prms.roi], 'wb') as f:
    pickle.dump(latlon_list, f)

219 glaciers in region 2 are included in this model run: ['00280', '00737', '00914', '01104', '01152', '01158', '01161', '01290', '01291', '01297', '01339', '01397', '01441', '01654', '01665', '01685', '01727', '01811', '01812', '01922', '01923', '02107', '02348', '02360', '02386', '02432', '02526', '02527', '02533', '02550', '02551', '02616', '02636', '02686', '02745', '02747', '02752', '02784', '02857', '02894', '02897', '02947', '02948', '02966', '03099', '03102', '03157', '03520', '03578', '03581'] and more
This study is focusing on 219 glaciers in region [2]
unique lat/lons: 108 




In [4]:
# ===== LOAD GLACIERS WITH DATA =====
main_glac_rgi_subset['mb_fn'] = np.nan
mb_binned_fp = debris_prms.mb_binned_fp

regions_str = [str(x).zfill(2) for x in debris_prms.roi_rgidict[debris_prms.roi]]

mb_fns = []
mb_rgiids = []
for i in os.listdir(mb_binned_fp):
    if i.endswith('_mb_bins.csv') and i.split('_')[0].split('.')[0].zfill(2) in regions_str:
        mb_fns.append(i)
        rgiid_raw = i.split('_')[0]
        rgiid = 'RGI60-' + rgiid_raw.split('.')[0].zfill(2) + '.' + rgiid_raw.split('.')[1]
        mb_rgiids.append(rgiid)
mb_rgiids = sorted(mb_rgiids)
mb_fns = sorted(mb_fns)
mb_fn_df = pd.DataFrame(np.zeros((len(mb_fns),2)), columns=['RGIId', 'mb_fn'])
mb_fn_df['RGIId'] = mb_rgiids
mb_fn_df['mb_fn'] = mb_fns

# Find glaciers that are debris-covered
mb_dc_rgiid = [value for value in list(mb_fn_df.RGIId.values) if value in list(main_glac_rgi_subset.RGIId.values)]
mb_fn_df_dc = mb_fn_df[mb_fn_df['RGIId'].isin(mb_dc_rgiid)]
mb_fn_df_dc = mb_fn_df_dc.sort_values('RGIId')

print('Debris-covered glaciers:', mb_fn_df_dc.shape[0], '\n\n')

mb_fn_dict = dict(zip(mb_fn_df_dc['RGIId'].values, mb_fn_df_dc['mb_fn'].values))

main_glac_rgi_subset['mb_fn'] = main_glac_rgi_subset.RGIId.map(mb_fn_dict)

Debris-covered glaciers: 176 




In [5]:
# ===== SELECT GLACIERS WITH DATA ====
main_glac_rgi_wobs = main_glac_rgi_subset.dropna(subset=['mb_fn']).copy()
# print('subset wdata length:', main_glac_rgi_wobs.shape)
main_glac_rgi_wobs.reset_index(inplace=True, drop=True)

# Update the latlon unique pickle files
latlon_nearidx_unique = sorted(list(set(main_glac_rgi_wobs['latlon_nearidx'].values)))
latlon_unique_dict = dict(zip(latlon_nearidx_unique,np.arange(0,len(latlon_nearidx_unique))))
latlon_unique_dict_reversed = dict(zip(np.arange(0,len(latlon_nearidx_unique)),latlon_nearidx_unique))
main_glac_rgi_wobs['latlon_unique_no'] = main_glac_rgi_wobs['latlon_nearidx'].map(latlon_unique_dict)

print('unique lat/lons:', len(np.unique(main_glac_rgi_wobs['latlon_unique_no'])), '\n\n')

lat_list = np.array([ds.latitude[x[0]].values for x in latlon_nearidx_unique])
lon_list = np.array([ds.longitude[x[1]].values for x in latlon_nearidx_unique])
latlon_list = list(tuple(zip(list(lat_list), list(lon_list))))

# Pickle unique lat/lons that will be used for melt model
with open(debris_prms.latlon_unique_fp + debris_prms.latlon_unique_dict[debris_prms.roi], 'wb') as f:
    pickle.dump(latlon_list, f)
    
main_glac_rgi_wobs

unique lat/lons: 93 




,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,...,TermType,Surging,RefDate,glacno,rgino_str,RGIId_float,CenLon_360,latlon_nearidx,latlon_unique_no,mb_fn
0,279,RGI60-02.00280,-122.57300,49.88620,2,2,5.938,1671,2340,1949,...,0,0,20059999,280,02.00280,2.00280,237.42700,"(160, 950)",80,2.00280_mb_bins.csv
1,736,RGI60-02.00737,-116.52800,50.21440,2,3,5.536,2237,3104,2586,...,0,0,20059999,737,02.00737,2.00737,243.47200,"(159, 974)",79,2.00737_mb_bins.csv
2,913,RGI60-02.00914,-123.85700,50.31570,2,2,4.372,1056,2233,1486,...,0,0,20059999,914,02.00914,2.00914,236.14300,"(159, 945)",77,2.00914_mb_bins.csv
3,1103,RGI60-02.01104,-122.60500,50.41950,2,2,3.016,1850,2539,2054,...,0,0,20059999,1104,02.01104,2.01104,237.39500,"(158, 950)",75,2.01104_mb_bins.csv
4,1151,RGI60-02.01152,-123.89000,50.34100,2,2,12.021,1280,2505,1908,...,0,0,20059999,1152,02.01152,2.01152,236.11000,"(159, 944)",76,2.01152_mb_bins.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,18764,RGI60-02.18765,-121.01578,48.31119,2,4,4.845,1523,2642,2125,...,0,0,19589999,18765,02.18765,2.18765,238.98422,"(167, 956)",84,2.18765_mb_bins.csv
172,18769,RGI60-02.18770,-121.00437,48.07234,2,4,2.125,1845,2490,2253,...,0,0,19849999,18770,02.18770,2.18770,238.99563,"(168, 956)",85,2.18770_mb_bins.csv
173,18777,RGI60-02.18778,-121.05735,48.35698,2,4,2.924,1613,2196,1891,...,0,0,19589999,18778,02.18778,2.18778,238.94265,"(167, 956)",84,2.18778_mb_bins.csv
174,18804,RGI60-02.18805,-109.63833,43.17324,2,5,3.061,3328,4052,3600,...,0,0,19669999,18805,02.18805,2.18805,250.36167,"(187, 1001)",91,2.18805_mb_bins.csv


In [6]:
# print('DELETE ME - HACK FOR DEVELOPMENT')
# print(np.where(main_glac_rgi_wobs['latlon_unique_no'] == 172)[0])
# main_glac_rgi_wobs = main_glac_rgi_wobs.loc[372:373,:]
# main_glac_rgi_wobs['mb_fn'].values

In [7]:
# ===== DEBRIS ELEVATION STATS ====================================================================================
# CALCULATE DEBRIS ELEVATION STATS FOR GLACIERS WITH DATA FOR EACH UNIQUE LAT/LON
elev_stats_latlon_dict = {}
latlon_list_updated = []
rgiid_4cal = []
for nlatlon, latlon_unique in enumerate(np.unique(main_glac_rgi_wobs.latlon_unique_no)):
# for nlatlon, latlon_unique in enumerate([np.unique(main_glac_rgi_wobs.latlon_unique_no)[0]]):

    main_glac_rgi_subset = main_glac_rgi_wobs[main_glac_rgi_wobs['latlon_unique_no'] == latlon_unique]
    main_glac_rgi_subset.reset_index(inplace=True, drop=True)
    
    # Debris elevation stats should be done by lat/lon
    df_all = None
    elev_list_all = []
    df_idx_count = 0
    count_width_passes = 0
    for nglac, glac_fn in enumerate(main_glac_rgi_subset.mb_fn.values):
#     for nglac, glac_fn in enumerate([main_glac_rgi_subset.mb_fn.values[0]]):

        glac_fullfn = debris_prms.mb_binned_fp + glac_fn
        
        glac_str_noleadzero = glac_fullfn.split('/')[-1].split('_')[0]
        rgiid = 'RGI60-' + glac_str_noleadzero.split('.')[0].zfill(2) + '.' + glac_str_noleadzero.split('.')[1]

        print(main_glac_rgi_subset.loc[nglac,'RGIId'], rgiid)
        
        # Select bins that meet calibratioin criteria
        df_raw = pd.read_csv(glac_fullfn)
        df = df_raw.dropna(subset=['mb_bin_mean_mwea'])
        df['z1_bin_areas_perc_cum'] = np.cumsum(df['z1_bin_area_valid_km2']) /df['z1_bin_area_valid_km2'].sum() * 100
        # add width to bins
        widths_fp = debris_prms.oggm_fp + 'widths/' + 'RGI60-' + rgiid.split('-')[1].split('.')[0] + '/'
        widths_fn = rgiid + '_widths_m.csv'
        try:
            # Add width to each elevation bin
            widths_df = pd.read_csv(widths_fp + widths_fn)
            elev_nearidx = (np.abs(df['bin_center_elev_m'].values[:,np.newaxis] - 
                                   widths_df['elev'].values).argmin(axis=1))
            df['width_m'] = widths_df.loc[elev_nearidx,'width_m'].values
        except:
            df['width_m'] = 0
        
        df_idx = np.where((df['vm_med'] <= debris_prms.vel_threshold) 
                          & (df['width_m'] >= debris_prms.width_min_dict[debris_prms.roi])
                          & (df['dc_bin_area_perc'] >= debris_prms.debrisperc_threshold)
                          & (df['dc_bin_count_valid'] >= 10)
                          & (df['z1_bin_areas_perc_cum'] <= debris_prms.term_area_perc)
                          )[0]
        df_debris = df.loc[df_idx,:]
        df_debris.reset_index(inplace=True, drop=True)
        df_idx_count += len(df_idx)
        
            
        if len(df_idx) > 0:
            for nelev, elev in enumerate(list(df_debris['bin_center_elev_m'].values)):
                elev_list_single = list(np.repeat(elev, df_debris.loc[nelev,'dc_bin_count_valid']))
                elev_list_all.extend(elev_list_single)
            
#             # only work with terminus
#             df_idx_dif = list(df_idx[1:] - df_idx[:-1])
#             if np.sum(df_idx_dif) == len(df_idx)-1:
#                 df_idx_nojump = df_idx
#             else:
#                 idx_jumpinbins = df_idx_dif.index(next(filter(lambda x: x>1, df_idx_dif)))
#                 df_idx_nojump = df_idx[0:idx_jumpinbins+1]
#             df_debris_nojump = df_debris.loc[df_idx_nojump,:]
#             df_debris_nojump.reset_index(inplace=True, drop=True)
#             # Median width to ensure terminus velocities can be estimated
#             width_median = np.median(widths_m[np.where(h < df_debris_nojump['bin_center_elev_m'].max())[0]])
#             if width_median > debris_prms.width_min_dict[debris_prms.roi]:
#                 for nelev, elev in enumerate(list(df_debris_nojump['bin_center_elev_m'].values)):
#                     elev_list_single = list(np.repeat(elev, df_debris_nojump.loc[nelev,'dc_bin_count_valid']))
#                     elev_list_all.extend(elev_list_single)
#                 count_width_passes += 1
    
            rgiid_4cal.append(rgiid.split('-')[1])
        
    if df_idx_count > 0:
        dc_zmean = np.mean(elev_list_all)
        dc_zstd = np.std(elev_list_all)
        dc_zmed = malib.fast_median(elev_list_all)
        dc_zmad = malib.mad(elev_list_all)
        
        lat_deg = float(ds.latitude[latlon_unique_dict_reversed[latlon_unique][0]].values)
        lon_deg = float(ds.longitude[latlon_unique_dict_reversed[latlon_unique][1]].values)
        elev_stats_latlon_dict[lat_deg,lon_deg] = [dc_zmean, dc_zstd, dc_zmed, dc_zmad]
        latlon_list_updated.append((lat_deg, lon_deg))
        
print('unique lat/lons updated:', len(latlon_list_updated))
# Update pickle of unique lat/lons that will be used for melt model
with open(debris_prms.latlon_unique_fp + debris_prms.latlon_unique_dict[debris_prms.roi], 'wb') as f:
    pickle.dump(latlon_list_updated, f)

RGI60-02.12368 RGI60-02.12368
RGI60-02.12266 RGI60-02.12266
RGI60-02.11861 RGI60-02.11861
RGI60-02.11836 RGI60-02.11836
RGI60-02.11134 RGI60-02.11134
RGI60-02.11119 RGI60-02.11119
RGI60-02.10972 RGI60-02.10972
RGI60-02.10988 RGI60-02.10988
RGI60-02.11040 RGI60-02.11040
RGI60-02.10599 RGI60-02.10599
RGI60-02.10355 RGI60-02.10355
RGI60-02.10363 RGI60-02.10363
RGI60-02.10468 RGI60-02.10468
RGI60-02.10126 RGI60-02.10126
RGI60-02.10176 RGI60-02.10176
RGI60-02.10071 RGI60-02.10071
RGI60-02.08364 RGI60-02.08364
RGI60-02.08465 RGI60-02.08465
RGI60-02.08547 RGI60-02.08547
RGI60-02.08762 RGI60-02.08762
RGI60-02.08509 RGI60-02.08509
RGI60-02.07938 RGI60-02.07938
RGI60-02.08309 RGI60-02.08309
RGI60-02.08310 RGI60-02.08310
RGI60-02.08282 RGI60-02.08282
RGI60-02.07778 RGI60-02.07778
RGI60-02.08298 RGI60-02.08298
RGI60-02.07701 RGI60-02.07701
RGI60-02.07783 RGI60-02.07783
RGI60-02.07525 RGI60-02.07525
RGI60-02.07235 RGI60-02.07235
RGI60-02.07300 RGI60-02.07300
RGI60-02.07384 RGI60-02.07384
RGI60-02.0

In [8]:
# Statistics of data coverage
rgiid_4cal = sorted(rgiid_4cal)
main_glac_rgi_4cal = debris_prms.selectglaciersrgitable(rgiid_4cal)
dc_area_dict = dict(zip(dc_shp.RGIId.values, dc_shp.DC_Area_v2.values))
main_glac_rgi_4cal['DC_Area_v2'] = main_glac_rgi_4cal.RGIId.map(dc_area_dict)
print('\nDC glaciers (used for cal):', main_glac_rgi_4cal.shape[0], 
      'DC Area (used for cal, km2):', main_glac_rgi_4cal.DC_Area_v2.sum() / 1e6)

158 glaciers in region 2 are included in this model run: ['00737', '00914', '01104', '01152', '01158', '01161', '01290', '01291', '01297', '01339', '01397', '01441', '01665', '01685', '01727', '01811', '01812', '01922', '01923', '02107', '02348', '02360', '02386', '02432', '02526', '02533', '02550', '02551', '02616', '02686', '02745', '02747', '02752', '02784', '02857', '02894', '02897', '02947', '02948', '02966', '03099', '03102', '03157', '03581', '03597', '03664', '03769', '03922', '03943', '03963'] and more
This study is focusing on 158 glaciers in region [2]

DC glaciers (used for cal): 158 DC Area (used for cal, km2): 112.328848


In [9]:
# ===== ADD DEBRIS ELEVATION STATS TO MET DATA ======
overwrite_dc_stats = True
for nlatlon, latlon in enumerate(latlon_list_updated):
# for nlatlon, latlon in enumerate([latlon_list_updated[0]]):
    
    lat_deg = latlon[0]
    lon_deg = latlon[1]
    
    print(nlatlon, lat_deg, lon_deg)
    
    if lat_deg < 0:
        lat_str = 'S-'
    else:
        lat_str = 'N-' 

    # ===== Meteorological data =====
    metdata_fn = debris_prms.metdata_fn_sample.replace(
        'XXXX', str(int(np.abs(lat_deg)*100)) + lat_str + str(int(lon_deg*100)) + 'E-')
    
    ds = xr.open_dataset(debris_prms.metdata_fp + metdata_fn) 
    try:
        print('  existed:', ds.dc_zmean.values, 'vs', elev_stats_latlon_dict[latlon][0])
    except:
        pass
    if 'dc_zmean' not in list(ds.keys()) or overwrite_dc_stats:
        # Add stats
        ds['dc_zmean'] = elev_stats_latlon_dict[latlon][0]
        ds['dc_zmean'].attrs = {'units':'m a.s.l.', 'long_name':'Mean debris cover elevation', 
                                'comment':'converted from debris cover with data that will be used for subdebris melt inversion'}
        ds['dc_zstd'] = elev_stats_latlon_dict[latlon][1]
        ds['dc_zstd'].attrs = {'units':'m a.s.l.', 'long_name':'Standard deviation of debris cover elevation', 
                               'comment':'converted from debris cover with data that will be used for subdebris melt inversion'}
        ds['dc_zmed'] = elev_stats_latlon_dict[latlon][2]
        ds['dc_zmed'].attrs = {'units':'m a.s.l.', 'long_name':'Median debris cover elevation', 
                               'comment':'converted from debris cover with data that will be used for subdebris melt inversion'}
        ds['dc_zmad'] = elev_stats_latlon_dict[latlon][3]
        ds['dc_zmad'].attrs = {'units':'m a.s.l.', 'long_name':'Median absolute deviation of debris cover elevation', 
                               'comment':'converted from debris cover with data that will be used for subdebris melt inversion'}

        try:
            ds.close()
        except:
            continue
            
        # Export updated dataset
        ds.to_netcdf(debris_prms.metdata_fp + metdata_fn, mode='a')
    else:
        print(lat_deg, lon_deg, 'exists')

0 58.5 234.5
  existed: 1871.2168674698796 vs 1871.2168674698796
1 58.25 235.25
  existed: 1966.0504201680671 vs 1966.0504201680671
2 57.5 235.75
  existed: 1978.960396039604 vs 1978.960396039604
3 57.25 236.0
  existed: 1753.5416666666667 vs 1753.5416666666667
4 54.75 230.5
  existed: 951.0390763765541 vs 951.0390763765541
5 54.75 230.75
6 54.5 230.5
  existed: 617.3177366702938 vs 617.3177366702938
7 54.5 230.75
  existed: 964.5454545454545 vs 964.5454545454545
8 54.0 239.5
  existed: 1817.9539295392954 vs 1817.9539295392954
9 54.0 239.75
  existed: 1832.7624309392265 vs 1832.7624309392265
10 53.75 232.0
  existed: 1211.025641025641 vs 1211.025641025641
11 53.75 232.25
  existed: 1285.110294117647 vs 1285.110294117647
12 53.0 232.5
  existed: 906.123595505618 vs 906.123595505618
13 53.0 232.75
  existed: 1034.6341463414635 vs 1034.6341463414635
14 53.0 233.0
  existed: 1471.4751958224542 vs 1471.4751958224542
15 53.0 239.5
  existed: 1976.6304347826087 vs 1976.6304347826087
16 53.0 2

In [11]:
print('DONE!')

DONE!


In [11]:
# ==== OLD FILE OF LOADIING MULTIPLE DATASETS =====
# # ===== LOAD GLACIERS WITH LARSEN DATA =====
# dc_shp_subset['larsen_fullfn'] = np.nan
# larsen_fullfn_dict = {}
# if 'larsen' in input.mb_datasets:
#     mb_summary = pd.read_csv(input.larsen_fp + input.larsen_fn)
    
#     # Find glaciers that are debris-covered
#     larsen_dc_rgiid = [value for value in list(mb_summary.RGIId.values) 
#                        if value in list(dc_shp_subset.RGIId.values)]

#     mb_summary_dc = mb_summary[mb_summary['RGIId'].isin(larsen_dc_rgiid)]
#     mb_summary_dc = mb_summary_dc.sort_values('RGIId')
#     mb_summary_dc.reset_index(inplace=True, drop=True)
#     mb_summary_dc.loc[mb_summary_dc['name'] == 'Maclaren', 'name'] = 'MacLaren'
#     mb_summary_dc.loc[mb_summary_dc['name'] == 'Tlikakila Fork', 'name'] = 'TlikakilaGlacierFork'
#     mb_summary_dc.loc[mb_summary_dc['name'] == 'Tlikakila N. Fork', 'name'] = 'TlikakilaNorthFork'
#     mb_summary_dc['larsen_fullfn'] = np.nan
    
#     for n, glac_name in enumerate(mb_summary_dc.name.values):
# #     for n, glac_name in enumerate([mb_summary_dc.name.values[47]]):
# #         print(n, glac_name)
            
#         glac_name = glac_name.replace(' ', '')
#         glac_fns = []
#         start_yr = []
#         end_yr = []
#         for i in os.listdir(input.larsen_binned_fp):
#             if i.startswith(glac_name):
#                 glac_fns.append(i)
#                 start_yr.append(i.split('.')[1][0:4])
#                 end_yr.append(i.split('.')[2][0:4])
                
#         if len(glac_fns) > 0:
#             yr_dif = np.array(end_yr).astype(int) - np.array(start_yr).astype(int)
#             mb_fn = glac_fns[np.where(yr_dif == yr_dif.max())[0][0]]
            
#             # ===== Process Larsen dataset =====
#             larsen_data_raw = np.genfromtxt(input.larsen_binned_fp + mb_fn, skip_header=3)
#             larsen_data_header = ['E', 'DZ', 'DZ25', 'DZ75', 'AAD', 'MassChange', 'MassBal', 'NumData']
#             larsen_data = pd.DataFrame(larsen_data_raw, columns=larsen_data_header)
#             larsen_data['std from DZ25'] = np.absolute(larsen_data['DZ'] - larsen_data['DZ25']) / 0.67
#             larsen_data['std from DZ75'] = np.absolute(larsen_data['DZ'] - larsen_data['DZ75']) / 0.67
#             larsen_data[' dhdt_bin_std_ma'] = (larsen_data['std from DZ25'] + larsen_data['std from DZ75']) / 2
#             larsen_data[' mb_bin_std_mwea'] = larsen_data[' dhdt_bin_std_ma'] * 900 / 1000
#             larsen_data['AAD'] = larsen_data['AAD'] / 1e6
#             larsen_data['startyear'] = int(mb_fn.split('.')[1][0:4])
#             larsen_data['endyear'] = int(mb_fn.split('.')[2][0:4])
#             larsen_data = larsen_data.rename({'E': '# bin_center_elev_m',
#                                               'DZ': ' dhdt_bin_mean_ma',
#                                               'MassBal': ' mb_bin_mean_mwea',
#                                               'AAD': ' z1_bin_area_valid_km2',
#                                              }, axis='columns')
#             new_fn = mb_summary_dc.loc[n,'RGIId'].split('-')[1][1:] + '_larsen_mb_bins.csv'
#             larsen_data.to_csv(input.larsen_binned_fp + new_fn, index=False)
            
#             mb_summary_dc.loc[n, 'larsen_fullfn'] = input.larsen_binned_fp + new_fn
            
#         else:
#             print(n, glac_name, 'has no file\n')

#     mb_summary_dc.dropna(subset=['larsen_fullfn'], inplace=True)
#     mb_summary_dc.reset_index(inplace=True, drop=True)
    
#     print('Larsen debris-covered glaciers:', mb_summary_dc.shape[0], '\n\n')
    
#     larsen_fullfn_dict = dict(zip(mb_summary_dc['RGIId'].values, mb_summary_dc['larsen_fullfn'].values))
# #     print(larsen_fullfn_dict)
#     dc_shp_subset['larsen_fullfn'] = dc_shp_subset.RGIId.map(larsen_fullfn_dict)

# # ===== LOAD GLACIERS WITH BRAUN DATA =====
# dc_shp_subset['braun_fullfn'] = np.nan
# braun_fullfn_dict = {}
# if 'braun' in input.mb_datasets:
#     mb_binned_fp = input.main_directory + '/../mb_data/Braun/binned_data/'
# #     mb_binned_fp = input.mb_binned_fp
    
#     mb_fns = []
#     braun_rgiids = []
#     for i in os.listdir(mb_binned_fp):
#         if i.endswith('_mb_bins.csv'):
#             mb_fns.append(mb_binned_fp + i)
#             rgiid_raw = i.split('_')[0]
#             rgiid = 'RGI60-' + rgiid_raw.split('.')[0].zfill(2) + '.' + rgiid_raw.split('.')[1]
#             braun_rgiids.append(rgiid)
#     braun_fn_df = pd.DataFrame(np.zeros((len(mb_fns),2)), columns=['RGIId', 'braun_fn'])
#     braun_fn_df['RGIId'] = braun_rgiids
#     braun_fn_df['braun_fullfn'] = mb_fns
    
#     # Find glaciers that are debris-covered
#     braun_dc_rgiid = [value for value in list(braun_fn_df.RGIId.values) 
#                        if value in list(dc_shp_subset.RGIId.values)]
#     braun_fn_df_dc = braun_fn_df[braun_fn_df['RGIId'].isin(braun_dc_rgiid)]
#     braun_fn_df_dc = braun_fn_df_dc.sort_values('RGIId')
    
#     print('Braun debris-covered glaciers:', braun_fn_df_dc.shape[0], '\n\n')
    
#     braun_fullfn_dict = dict(zip(braun_fn_df_dc['RGIId'].values, braun_fn_df_dc['braun_fullfn'].values))
    
#     dc_shp_subset['braun_fullfn'] = dc_shp_subset.RGIId.map(braun_fullfn_dict)

# # ===== LOAD GLACIERS WITH SHEAN DATA =====
# dc_shp_subset['shean_fullfn'] = np.nan
# shean_fullfn_dict = {}
# if 'shean' in input.mb_datasets:
# #     mb_binned_fp = input.main_directory + '/../mb_data/Shean_2019_0213/mb_combined_20190213_nmad_bins/'
#     mb_binned_fp = input.mb_binned_fp
    
#     mb_fns = []
#     rgiids = []
#     for i in os.listdir(mb_binned_fp):
#         if i.endswith('_mb_bins.csv'):
#             mb_fns.append(mb_binned_fp + i)
#             rgiid_raw = i.split('_')[0]
#             rgiid = 'RGI60-' + rgiid_raw.split('.')[0].zfill(2) + '.' + rgiid_raw.split('.')[1]
#             rgiids.append(rgiid)
#     mb_fn_df = pd.DataFrame(np.zeros((len(mb_fns),2)), columns=['RGIId', 'mb_fn'])
#     mb_fn_df['RGIId'] = rgiids
#     mb_fn_df['mb_fullfn'] = mb_fns
    
#     # Find glaciers that are debris-covered
#     mb_dc_rgiid = [value for value in list(mb_fn_df.RGIId.values) 
#                    if value in list(dc_shp_subset.RGIId.values)]
#     mb_fn_df_dc = mb_fn_df[mb_fn_df['RGIId'].isin(mb_dc_rgiid)]
#     mb_fn_df_dc = mb_fn_df_dc.sort_values('RGIId')
    
#     print('shean debris-covered glaciers:', mb_fn_df_dc.shape[0], '\n\n')
    
#     shean_fullfn_dict = dict(zip(mb_fn_df_dc['RGIId'].values, mb_fn_df_dc['mb_fullfn'].values))
# #     print(shea_fullfn_dict)
#     dc_shp_subset['shean_fullfn'] = dc_shp_subset.RGIId.map(shean_fullfn_dict)

# # Merge dictionaries together
# mb_fn_dict = dict(list(larsen_fullfn_dict.items()) + list(braun_fullfn_dict.items()) + 
#                   list(shean_fullfn_dict.items()))